# Equal-Weight S&P 500 Index Fund

March 2025

Leonardo Zhao

## Introduction

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has almost US$600 billion (as of March 2025) in assets under management.

The goal of this project is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to create an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the libraries that we'll be using.

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import xlsxwriter

## Fetch List of Stock

Without using an API key, we will import all 503 stocks in the S&P 500, as listed on Wikipedia (as of March 2025).

The correctness of our fetch depends on the layout of the Wikipedia page. Let's hope they don't change it.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_table = tables[0] # assuming the first table stores S&P 500 data
sp500_tickers = list(sp500_table['Symbol'].str.replace(".", "-", regex=False))

## Create DataFrame of Prices

The next thing we need to do is add our stocks' price to a pandas DataFrame.

In [3]:
prices_df = yf.download(sp500_tickers, period="1d", progress=False)["Close"]
prices = prices_df.iloc[-1].dropna().to_dict()

YF.download() has changed argument auto_adjust default to True


In [4]:
sp500_df = pd.DataFrame({
    "Ticker": prices.keys(),
    "Current Price": prices.values(),
    "Number of Shares to Buy": "N/A"
})

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [5]:
correct_input = False
while not correct_input:
    try:
        portfolio_size = input("Enter the value of your portfolio (in million):")
        portfolio_value = float(portfolio_size) * 1000000  # Convert directly here
        correct_input = True
    except ValueError:
        print("That's not a number! Please try again.")

Enter the value of your portfolio (in million): 37.5342


In [6]:
position_size = float(portfolio_value) / len(sp500_df.index)

for i in range (len(sp500_df.index)):
    sp500_df.loc[i, 'Number of Shares to Buy'] = np.floor(position_size / sp500_df['Current Price'][i])

print(position_size)
sp500_df

74620.675944334


,Ticker,Current Price,Number of Shares to Buy
0,A,116.940002,638.0
1,AAPL,218.229996,341.0
2,ABBV,205.740005,362.0
3,ABNB,120.544998,619.0
4,ABT,131.399994,567.0
...,...,...,...
498,XYL,120.260101,620.0
499,YUM,155.955002,478.0
500,ZBH,112.589996,662.0
501,ZBRA,280.470001,266.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [7]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
sp500_df.to_excel(writer, sheet_name='Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [8]:
dark_gray = '#808080'
light_gray = '#D3D3D3'
font_color = '#000000'

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'border': 1
    }
)

# Define format for integers (for Number of Shares to Buy)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'border': 1
    }
)

# Define format for strings (e.g., for Ticker column)
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

In [9]:
column_formats = { 
    'A': ['Ticker', string_format],
    'B': ['Current Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

for i, row in sp500_df.iterrows():
    row_color = light_gray if i % 2 == 0 else dark_gray
    writer.sheets['Recommended Trades'].write(f'A{i+2}', row['Ticker'], writer.book.add_format({'bg_color': row_color, 'font_color': font_color, 'border': 1}))
    writer.sheets['Recommended Trades'].write(f'B{i+2}', row['Current Price'], writer.book.add_format({'bg_color': row_color, 'font_color': font_color, 'border': 1, 'num_format': '$0.00'}))
    writer.sheets['Recommended Trades'].write(f'C{i+2}', row['Number of Shares to Buy'], writer.book.add_format({'bg_color': row_color, 'font_color': font_color, 'border': 1, 'num_format': '0'}))


## Saving Our Excel Output

Saving our Excel file is very easy:

In [10]:
writer.close()